# AsyncWeaviateClient Demo

In [1]:
from ray_retriever.serve.async_weaviate_client import AsyncWeaviateClient

## Setup client

In [2]:
aclient = AsyncWeaviateClient('127.0.0.1', 9001)

## Get schema

In [3]:
schema = await aclient.get_schema('Wikipedia')
print(schema)

{'class': 'Wikipedia', 'description': 'Class for Wikipedia', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'multiTenancyConfig': {'enabled': False}, 'properties': [{'dataType': ['text'], 'description': 'Text property', 'indexFilterable': True, 'indexSearchable': True, 'name': 'text', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'The ref_doc_id of the Node', 'indexFilterable': True, 'indexSearchable': True, 'name': 'ref_doc_id', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'node_info (in JSON)', 'indexFilterable': True, 'indexSearchable': True, 'name': 'node_info', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'The relationships of the node (in JSON)', 'indexFilterable': True, 'indexSearchable': True, 'name': 'relationships', 'tokenization': 'word'}, {'dataType': ['text'], 'description': "This property was generated by Weaviate

## Get all nodes

In [10]:
all_nodes = await aclient.get_all_nodes('Wikipedia', return_embeddings=True, max_result_size=100)
print(len(all_nodes))
print(all_nodes[0])

100
id='000108f1-3d15-4fd9-a7f1-d25f7440fd90' index_name='Wikipedia' metadata={'uri': 'https://simple.wikipedia.org/wiki/2010', 'title': '2010', 'source': 'wikipedia-20220301.simple'} text="2010\nApril \n\n April 3 – South African far-right leader Eugene Terre'Blanche is beaten to death at his farm near Ventersdorp, South Africa.\n April 3 – The African Renaissance Monument opens in Dakar, to mark 50 years of Senegal's independence.\n April 7 – Amidst fierce rioting in Kyrgyzstan, President Kurmanbek Bakiyev flees the capital city, Bishkek by plane to the southern city of Osh; the opposition seizes control of the government, and places former foreign minister Roza Otunbayeva as head of an interim government. She later becomes President on July 3.\n April 10 – Poland's President Lech Kaczynski and his wife Maria are among 96 people killed when their plane crashes in thick fog near Smolensk in western Russia.\n April 14 – A 6.9 magnitude Earthquake strikes Qinghai province in China, kill

## Get node by title

In [6]:
filter = {
    "path": "title",
    "operator": "Equal",
    "valueText": 'Alan Turing'
}

alan_turing_nodes = await aclient.get_all_nodes('Wikipedia', filter=filter, max_result_size=50)
print(len(alan_turing_nodes))
for node in alan_turing_nodes:
    print(node.text)

3
Alan Turing
Alas, Turing accidentally or otherwise lost his life in 1954, having been subjected by a British court to chemical castration, thus avoiding a custodial sentence. He is known to have ended his life at the age of 41 years, by eating an apple laced with cyanide.

Career 
Turing was one of the people who worked on the first computers. He created the theoretical  Turing machine in 1936. The machine was imaginary, but it included the idea of a computer program.

Turing was interested in artificial intelligence. He proposed the Turing test, to say when a machine could be called "intelligent". A computer could be said to "think" if a human talking with it could not tell it was a machine.

During World War II, Turing worked with others to break German ciphers (secret messages). He  worked for the Government Code and Cypher School (GC&CS) at Bletchley Park, Britain's codebreaking centre that produced Ultra intelligence.
Using cryptanalysis, he helped to break the codes of the Enig

## Get node by ID

In [7]:
filter = {
    "path": "id",
    "operator": "Equal",
    "valueText": all_nodes[0].id
}

single_node = await aclient.get_all_nodes('Wikipedia', 
                                          filter=filter, 
                                          max_result_size=10, 
                                          return_embeddings=True)
print(len(single_node))
print(single_node[0].id)

1
00c92c0c-815b-4bea-b4bc-d8b0375631df


## Get nodes by similarity

In [16]:
similar_nodes = await aclient.find_similar_nodes('Wikipedia',
                                 single_node[0].embedding, 
                                 similarity_top_k=3)

assert similar_nodes.nodes[0].id == single_node[0].id
assert abs(1.0 - similar_nodes.similarities[0]) < 0.001